In [61]:
import pandas as pd
import numpy as np

from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K

from sklearn.model_selection import StratifiedKFold

In [14]:
data = pd.read_csv("../data/image_file_features.csv", delimiter='\t', index_col=False)

data = data.drop("Unnamed: 0", axis=1)

data.head()

,path,h,w,pixels,aspect_ratio,file_size,pix_pb,r_mean,g_mean,b_mean,r_accum_err,g_accum_err,b_accum_err
0,../data/imgs/28491_68.jpg,800,800,640000,1.000000,85320,7.501172,229.413972,227.713289,212.647266,0.026861,0.036141,0.053210
1,../data/imgs/11997_101.jpg,800,800,640000,1.000000,166938,3.833759,126.274078,149.567391,155.617066,0.054656,0.036767,0.035571
2,../data/imgs/68542_80.jpg,634,793,502762,1.250789,157616,3.189790,180.925826,158.143263,141.555036,0.015865,0.030551,0.031391
3,../data/imgs/79412_25.jpg,800,800,640000,1.000000,151186,4.233196,216.552556,186.867486,176.672427,0.032652,0.038144,0.046619
4,../data/imgs/25196_117.jpg,462,600,277200,1.298701,51651,5.366789,179.549928,135.804163,100.765664,0.022439,0.034735,0.061929


In [18]:
data.loc[:, "label"] = data.loc[:, "path"].apply(lambda x: int(x.split("_")[1].split(".")[0]) - 1)

print(len(data))
data.head()

152221


,path,h,w,pixels,aspect_ratio,file_size,pix_pb,r_mean,g_mean,b_mean,r_accum_err,g_accum_err,b_accum_err,label
0,../data/imgs/28491_68.jpg,800,800,640000,1.000000,85320,7.501172,229.413972,227.713289,212.647266,0.026861,0.036141,0.053210,67
1,../data/imgs/11997_101.jpg,800,800,640000,1.000000,166938,3.833759,126.274078,149.567391,155.617066,0.054656,0.036767,0.035571,100
2,../data/imgs/68542_80.jpg,634,793,502762,1.250789,157616,3.189790,180.925826,158.143263,141.555036,0.015865,0.030551,0.031391,79
3,../data/imgs/79412_25.jpg,800,800,640000,1.000000,151186,4.233196,216.552556,186.867486,176.672427,0.032652,0.038144,0.046619,24
4,../data/imgs/25196_117.jpg,462,600,277200,1.298701,51651,5.366789,179.549928,135.804163,100.765664,0.022439,0.034735,0.061929,116


In [19]:
data = data[data.loc[:, "h"] > 50]
data = data[data.loc[:, "w"] > 50]
data = data[data.loc[:, "file_size"] > 1000]

print(len(data))

152036


In [30]:
all_y = data.loc[:, "label"].values
all_x = data.loc[:, ["h", "w", "pixels", "aspect_ratio", "file_size", "pix_pb", "r_mean", "g_mean", "b_mean", "r_accum_err", "g_accum_err", "b_accum_err"]].values

In [32]:
skf = StratifiedKFold(n_splits=10)

data_dict = {}

split = 0
for train_index, test_index in skf.split(all_x, all_y):
    split += 1
    data_dict["X_train_{:}".format(split)] = all_x[train_index]
    data_dict["y_train_{:}".format(split)] = all_y[train_index]
    data_dict["X_test_{:}".format(split)] = all_x[test_index]
    data_dict["y_test_{:}".format(split)] = all_y[test_index]
    
# X_train, X_test = X[train_index], X[test_index]

### Create a model

In [82]:
LR = 0.005
DECAY = 1e-4
DO = 0.15
EPOCHS = 250
MINI_TRAINS = 10

X_train = data_dict["X_train_1"]
y_train = to_categorical(data_dict["y_train_1"], 128)
X_test = data_dict["X_test_1"]
y_test = to_categorical(data_dict["y_test_1"], 128)

_input = Input(shape=(12,))
x = Dense(128)(_input)
x = BatchNormalization()(x)
x = Dropout(DO)(x)
x = Activation("relu")(x)

x = Dense(256)(x)
x = BatchNormalization()(x)
x = Dropout(DO)(x)
x = Activation("relu")(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = Dropout(DO)(x)
x = Activation("relu")(x)

output = Dense(128, activation='softmax')(x)
model = Model(inputs=_input, outputs=output)

OPTIMIZER = Adam(lr=LR, decay=DECAY)

model.compile(optimizer=OPTIMIZER, loss="categorical_crossentropy", metrics=["accuracy"])

for idx in range(MINI_TRAINS):
    temp_lr = LR / (2 * (idx + 1))
    
    K.set_value(OPTIMIZER.lr, temp_lr)
    
#     print("\nMini-Train: {:3d} Learning Rate: {:2.7f}".format((idx + 1), temp_lr))
    
    model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=EPOCHS, batch_size=1024, verbose=2)

    score = model.evaluate(X_test, y_test, batch_size=1024, verbose=0)
    
    print("Mini-Train: {:3d} Test Accuracy: {:2.2f}% Learning Rate: {:2.7f}".format((idx + 1), score[1] * 100, temp_lr))

Train on 136770 samples, validate on 15266 samples
Epoch 1/250
 - 6s - loss: 4.5342 - acc: 0.0321 - val_loss: 4.4662 - val_acc: 0.0362
Epoch 2/250
 - 2s - loss: 4.4522 - acc: 0.0371 - val_loss: 4.4332 - val_acc: 0.0426
Epoch 3/250
 - 2s - loss: 4.4333 - acc: 0.0386 - val_loss: 4.4215 - val_acc: 0.0429
Epoch 4/250
 - 2s - loss: 4.4243 - acc: 0.0398 - val_loss: 4.4135 - val_acc: 0.0391
Epoch 5/250
 - 2s - loss: 4.4171 - acc: 0.0414 - val_loss: 4.4170 - val_acc: 0.0428
Epoch 6/250
 - 2s - loss: 4.4122 - acc: 0.0425 - val_loss: 4.4076 - val_acc: 0.0452
Epoch 7/250
 - 2s - loss: 4.4061 - acc: 0.0424 - val_loss: 4.4097 - val_acc: 0.0430
Epoch 8/250
 - 2s - loss: 4.4021 - acc: 0.0432 - val_loss: 4.4276 - val_acc: 0.0427
Epoch 9/250
 - 2s - loss: 4.3963 - acc: 0.0453 - val_loss: 4.4695 - val_acc: 0.0409
Epoch 10/250
 - 2s - loss: 4.3786 - acc: 0.0488 - val_loss: 4.4646 - val_acc: 0.0381
Epoch 11/250
 - 2s - loss: 4.3542 - acc: 0.0526 - val_loss: 4.4326 - val_acc: 0.0399
Epoch 12/250
 - 2s - lo

Epoch 97/250
 - 2s - loss: 4.1155 - acc: 0.0842 - val_loss: 4.2362 - val_acc: 0.0690
Epoch 98/250
 - 2s - loss: 4.1143 - acc: 0.0845 - val_loss: 4.2488 - val_acc: 0.0688
Epoch 99/250
 - 2s - loss: 4.1118 - acc: 0.0850 - val_loss: 4.4884 - val_acc: 0.0486
Epoch 100/250
 - 2s - loss: 4.1178 - acc: 0.0832 - val_loss: 4.2288 - val_acc: 0.0689
Epoch 101/250
 - 2s - loss: 4.1168 - acc: 0.0841 - val_loss: 4.2409 - val_acc: 0.0672
Epoch 102/250
 - 2s - loss: 4.1094 - acc: 0.0846 - val_loss: 4.3153 - val_acc: 0.0586
Epoch 103/250
 - 2s - loss: 4.1051 - acc: 0.0857 - val_loss: 4.2408 - val_acc: 0.0679
Epoch 104/250
 - 2s - loss: 4.1064 - acc: 0.0856 - val_loss: 4.2886 - val_acc: 0.0614
Epoch 105/250
 - 2s - loss: 4.1082 - acc: 0.0858 - val_loss: 4.2705 - val_acc: 0.0607
Epoch 106/250
 - 2s - loss: 4.1054 - acc: 0.0861 - val_loss: 4.2641 - val_acc: 0.0631
Epoch 107/250
 - 2s - loss: 4.1035 - acc: 0.0854 - val_loss: 4.3137 - val_acc: 0.0565
Epoch 108/250
 - 2s - loss: 4.1045 - acc: 0.0857 - val_lo

Epoch 193/250
 - 2s - loss: 4.0529 - acc: 0.0928 - val_loss: 4.1414 - val_acc: 0.0810
Epoch 194/250
 - 2s - loss: 4.0514 - acc: 0.0930 - val_loss: 4.1859 - val_acc: 0.0739
Epoch 195/250
 - 2s - loss: 4.0495 - acc: 0.0933 - val_loss: 4.2280 - val_acc: 0.0677
Epoch 196/250
 - 2s - loss: 4.0501 - acc: 0.0926 - val_loss: 4.3063 - val_acc: 0.0606
Epoch 197/250
 - 2s - loss: 4.0507 - acc: 0.0929 - val_loss: 4.2573 - val_acc: 0.0645
Epoch 198/250
 - 2s - loss: 4.0507 - acc: 0.0931 - val_loss: 4.2534 - val_acc: 0.0652
Epoch 199/250
 - 2s - loss: 4.0492 - acc: 0.0934 - val_loss: 4.1379 - val_acc: 0.0795
Epoch 200/250
 - 2s - loss: 4.0467 - acc: 0.0938 - val_loss: 4.2043 - val_acc: 0.0716
Epoch 201/250
 - 2s - loss: 4.0477 - acc: 0.0933 - val_loss: 4.1979 - val_acc: 0.0709
Epoch 202/250
 - 2s - loss: 4.0487 - acc: 0.0936 - val_loss: 4.1317 - val_acc: 0.0833
Epoch 203/250
 - 2s - loss: 4.0469 - acc: 0.0940 - val_loss: 4.1393 - val_acc: 0.0822
Epoch 204/250
 - 2s - loss: 4.0476 - acc: 0.0928 - val

Epoch 38/250
 - 2s - loss: 4.0192 - acc: 0.0972 - val_loss: 4.1049 - val_acc: 0.0856
Epoch 39/250
 - 2s - loss: 4.0184 - acc: 0.0974 - val_loss: 4.1106 - val_acc: 0.0850
Epoch 40/250
 - 2s - loss: 4.0208 - acc: 0.0962 - val_loss: 4.1074 - val_acc: 0.0855
Epoch 41/250
 - 2s - loss: 4.0184 - acc: 0.0975 - val_loss: 4.0901 - val_acc: 0.0896
Epoch 42/250
 - 2s - loss: 4.0189 - acc: 0.0981 - val_loss: 4.0869 - val_acc: 0.0872
Epoch 43/250
 - 2s - loss: 4.0174 - acc: 0.0983 - val_loss: 4.1289 - val_acc: 0.0842
Epoch 44/250
 - 2s - loss: 4.0196 - acc: 0.0983 - val_loss: 4.1288 - val_acc: 0.0846
Epoch 45/250
 - 2s - loss: 4.0176 - acc: 0.0977 - val_loss: 4.0896 - val_acc: 0.0888
Epoch 46/250
 - 2s - loss: 4.0178 - acc: 0.0984 - val_loss: 4.1044 - val_acc: 0.0865
Epoch 47/250
 - 2s - loss: 4.0180 - acc: 0.0980 - val_loss: 4.0833 - val_acc: 0.0909
Epoch 48/250
 - 2s - loss: 4.0168 - acc: 0.0967 - val_loss: 4.1547 - val_acc: 0.0804
Epoch 49/250
 - 2s - loss: 4.0176 - acc: 0.0973 - val_loss: 4.115

Epoch 134/250
 - 2s - loss: 4.0106 - acc: 0.0982 - val_loss: 4.0910 - val_acc: 0.0894
Epoch 135/250
 - 2s - loss: 4.0052 - acc: 0.0994 - val_loss: 4.1140 - val_acc: 0.0851
Epoch 136/250
 - 2s - loss: 4.0086 - acc: 0.0993 - val_loss: 4.0741 - val_acc: 0.0905
Epoch 137/250
 - 2s - loss: 4.0100 - acc: 0.0990 - val_loss: 4.1020 - val_acc: 0.0884
Epoch 138/250
 - 2s - loss: 4.0088 - acc: 0.0990 - val_loss: 4.0664 - val_acc: 0.0920
Epoch 139/250
 - 2s - loss: 4.0108 - acc: 0.0991 - val_loss: 4.1162 - val_acc: 0.0851
Epoch 140/250
 - 2s - loss: 4.0067 - acc: 0.0999 - val_loss: 4.0891 - val_acc: 0.0887
Epoch 141/250
 - 2s - loss: 4.0079 - acc: 0.0984 - val_loss: 4.1002 - val_acc: 0.0876
Epoch 142/250
 - 2s - loss: 4.0085 - acc: 0.0994 - val_loss: 4.0935 - val_acc: 0.0890
Epoch 143/250
 - 2s - loss: 4.0069 - acc: 0.0991 - val_loss: 4.1217 - val_acc: 0.0846
Epoch 144/250
 - 2s - loss: 4.0105 - acc: 0.0980 - val_loss: 4.0989 - val_acc: 0.0878
Epoch 145/250
 - 2s - loss: 4.0065 - acc: 0.0991 - val

...

Epoch 106/250
 - 2s - loss: 3.9742 - acc: 0.1041 - val_loss: 4.0466 - val_acc: 0.0950
Epoch 107/250
 - 2s - loss: 3.9765 - acc: 0.1036 - val_loss: 4.0430 - val_acc: 0.0970
Epoch 108/250
 - 2s - loss: 3.9769 - acc: 0.1026 - val_loss: 4.0439 - val_acc: 0.0961
Epoch 109/250
 - 2s - loss: 3.9768 - acc: 0.1040 - val_loss: 4.0420 - val_acc: 0.0963
Epoch 110/250
 - 2s - loss: 3.9763 - acc: 0.1031 - val_loss: 4.0429 - val_acc: 0.0964
Epoch 111/250
 - 2s - loss: 3.9741 - acc: 0.1033 - val_loss: 4.0427 - val_acc: 0.0967
Epoch 112/250
 - 2s - loss: 3.9765 - acc: 0.1036 - val_loss: 4.0429 - val_acc: 0.0956
Epoch 113/250
 - 2s - loss: 3.9752 - acc: 0.1018 - val_loss: 4.0411 - val_acc: 0.0956
Epoch 114/250
 - 2s - loss: 3.9762 - acc: 0.1034 - val_loss: 4.0451 - val_acc: 0.0963
Epoch 115/250
 - 2s - loss: 3.9776 - acc: 0.1037 - val_loss: 4.0412 - val_acc: 0.0963
Epoch 116/250
 - 2s - loss: 3.9750 - acc: 0.1038 - val_loss: 4.0432 - val_acc: 0.0968
Epoch 117/250
 - 2s - loss: 3.9765 - acc: 0.1036 - val

Epoch 202/250
 - 2s - loss: 3.9784 - acc: 0.1032 - val_loss: 4.0459 - val_acc: 0.0956
Epoch 203/250
 - 2s - loss: 3.9781 - acc: 0.1035 - val_loss: 4.0431 - val_acc: 0.0959
Epoch 204/250
 - 2s - loss: 3.9759 - acc: 0.1036 - val_loss: 4.0446 - val_acc: 0.0949
Epoch 205/250
 - 2s - loss: 3.9768 - acc: 0.1033 - val_loss: 4.0426 - val_acc: 0.0960
Epoch 206/250
 - 2s - loss: 3.9766 - acc: 0.1039 - val_loss: 4.0456 - val_acc: 0.0955
Epoch 207/250
 - 2s - loss: 3.9738 - acc: 0.1038 - val_loss: 4.0417 - val_acc: 0.0966
Epoch 208/250
 - 2s - loss: 3.9763 - acc: 0.1033 - val_loss: 4.0425 - val_acc: 0.0965
Epoch 209/250
 - 2s - loss: 3.9788 - acc: 0.1031 - val_loss: 4.0446 - val_acc: 0.0962
Epoch 210/250
 - 2s - loss: 3.9740 - acc: 0.1034 - val_loss: 4.0425 - val_acc: 0.0964
Epoch 211/250
 - 2s - loss: 3.9768 - acc: 0.1032 - val_loss: 4.0412 - val_acc: 0.0960
Epoch 212/250
 - 2s - loss: 3.9784 - acc: 0.1032 - val_loss: 4.0412 - val_acc: 0.0973
Epoch 213/250
 - 2s - loss: 3.9758 - acc: 0.1030 - val

Epoch 47/250
 - 2s - loss: 3.9761 - acc: 0.1030 - val_loss: 4.0414 - val_acc: 0.0960
Epoch 48/250
 - 2s - loss: 3.9758 - acc: 0.1034 - val_loss: 4.0428 - val_acc: 0.0956
Epoch 49/250
 - 2s - loss: 3.9755 - acc: 0.1043 - val_loss: 4.0433 - val_acc: 0.0960
Epoch 50/250
 - 2s - loss: 3.9755 - acc: 0.1031 - val_loss: 4.0454 - val_acc: 0.0958
Epoch 51/250
 - 2s - loss: 3.9769 - acc: 0.1040 - val_loss: 4.0471 - val_acc: 0.0954
Epoch 52/250
 - 2s - loss: 3.9758 - acc: 0.1042 - val_loss: 4.0437 - val_acc: 0.0972
Epoch 53/250
 - 2s - loss: 3.9764 - acc: 0.1034 - val_loss: 4.0438 - val_acc: 0.0967
Epoch 54/250
 - 2s - loss: 3.9759 - acc: 0.1037 - val_loss: 4.0430 - val_acc: 0.0958
Epoch 55/250
 - 2s - loss: 3.9760 - acc: 0.1032 - val_loss: 4.0462 - val_acc: 0.0958
Epoch 56/250
 - 2s - loss: 3.9749 - acc: 0.1029 - val_loss: 4.0429 - val_acc: 0.0964
Epoch 57/250
 - 2s - loss: 3.9766 - acc: 0.1037 - val_loss: 4.0418 - val_acc: 0.0967
Epoch 58/250
 - 2s - loss: 3.9759 - acc: 0.1036 - val_loss: 4.048

Epoch 143/250
 - 2s - loss: 3.9743 - acc: 0.1033 - val_loss: 4.0458 - val_acc: 0.0953
Epoch 144/250
 - 2s - loss: 3.9758 - acc: 0.1042 - val_loss: 4.0482 - val_acc: 0.0955
Epoch 145/250
 - 2s - loss: 3.9759 - acc: 0.1043 - val_loss: 4.0413 - val_acc: 0.0967
Epoch 146/250
 - 2s - loss: 3.9760 - acc: 0.1038 - val_loss: 4.0429 - val_acc: 0.0958
Epoch 147/250
 - 2s - loss: 3.9751 - acc: 0.1028 - val_loss: 4.0439 - val_acc: 0.0956
Epoch 148/250
 - 2s - loss: 3.9766 - acc: 0.1038 - val_loss: 4.0467 - val_acc: 0.0954
Epoch 149/250
 - 2s - loss: 3.9774 - acc: 0.1036 - val_loss: 4.0437 - val_acc: 0.0967
Epoch 150/250
 - 2s - loss: 3.9782 - acc: 0.1041 - val_loss: 4.0447 - val_acc: 0.0957
Epoch 151/250
 - 2s - loss: 3.9778 - acc: 0.1034 - val_loss: 4.0444 - val_acc: 0.0958
Epoch 152/250
 - 2s - loss: 3.9755 - acc: 0.1036 - val_loss: 4.0407 - val_acc: 0.0962
Epoch 153/250
 - 2s - loss: 3.9745 - acc: 0.1038 - val_loss: 4.0427 - val_acc: 0.0958
Epoch 154/250
 - 2s - loss: 3.9755 - acc: 0.1034 - val

Epoch 239/250
 - 2s - loss: 3.9758 - acc: 0.1044 - val_loss: 4.0424 - val_acc: 0.0951
Epoch 240/250
 - 2s - loss: 3.9768 - acc: 0.1037 - val_loss: 4.0466 - val_acc: 0.0958
Epoch 241/250
 - 2s - loss: 3.9758 - acc: 0.1039 - val_loss: 4.0430 - val_acc: 0.0962
Epoch 242/250
 - 2s - loss: 3.9732 - acc: 0.1032 - val_loss: 4.0401 - val_acc: 0.0958
Epoch 243/250
 - 2s - loss: 3.9775 - acc: 0.1029 - val_loss: 4.0427 - val_acc: 0.0964
Epoch 244/250
 - 2s - loss: 3.9768 - acc: 0.1034 - val_loss: 4.0406 - val_acc: 0.0956
Epoch 245/250
 - 2s - loss: 3.9752 - acc: 0.1039 - val_loss: 4.0444 - val_acc: 0.0950
Epoch 246/250
 - 2s - loss: 3.9765 - acc: 0.1038 - val_loss: 4.0421 - val_acc: 0.0960
Epoch 247/250
 - 2s - loss: 3.9749 - acc: 0.1035 - val_loss: 4.0430 - val_acc: 0.0966
Epoch 248/250
 - 2s - loss: 3.9735 - acc: 0.1045 - val_loss: 4.0430 - val_acc: 0.0963
Epoch 249/250
 - 2s - loss: 3.9764 - acc: 0.1037 - val_loss: 4.0419 - val_acc: 0.0960
Epoch 250/250
 - 2s - loss: 3.9750 - acc: 0.1033 - val

In [70]:
from sklearn.linear_model import LogisticRegression

In [75]:
logit = LogisticRegression()

logit.fit(X_train, data_dict["y_train_1"])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
score = logit.score(X_test, data_dict["y_test_1"])

print(score)

0.04146469278134416


In [78]:
logit2 = LogisticRegression(C=1e5)

logit2.fit(X_train, data_dict["y_train_1"])
score = logit2.score(X_test, data_dict["y_test_1"])

print(score)


0.04100615747412551
